In [ ]:
import pandas as pd
import plotly
import plotly.graph_objects as go
import networkx as nx
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import SpectralClustering
from copy import copy
from sklearn.metrics import accuracy_score
plotly.offline.init_notebook_mode(connected = True)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
train = pd.read_csv('Training_wells.csv')

In [ ]:
# дропаем колонки, в которых есть хотя бы один нан
dropList_numeric = []
dropList_categorial = []
for colname, colvalue in zip(initial_dataset.columns, initial_dataset.isnull().any()):
    if colvalue == True and np.issubdtype(initial_dataset[colname].dtype, np.number) == True:
        dropList_numeric.append(colname)
    elif colvalue == True and not np.issubdtype(initial_dataset[colname].dtype, np.number):
        dropList_categorial.append(colname)
dataset = initial_dataset.drop(dropList_numeric + dropList_categorial, axis = 1)

# записываем координаты в отдельные переменные и выбрасываем их из датасета
latitude = dataset['7. Latitude']
longitude = dataset['8. Longitude']
dataset = dataset.drop(['7. Latitude', '8. Longitude', '9. Operator company'], axis = 1)
dataset = dataset.drop(dataset.columns[1:6], axis = 1) # ещё выбросимненужные данные


In [ ]:
# выделяем категориальные и непрерывные признаки
categorial_parameters = []
numeric_parameters = []
for column in dataset.columns[1:]:
    if np.issubdtype(dataset[column].dtype, np.number):
        numeric_parameters.append(column)
    elif dataset[column].dtype == object:
        categorial_parameters.append(column)

In [ ]:
# шифруем лейбл-энкодером все категориальные признаки
le = LabelEncoder()
for cat_col in categorial_parameters:
    dataset[cat_col] = le.fit_transform(dataset[cat_col])
    initial_dataset[cat_col] = le.fit_transform(initial_dataset[cat_col])
dataset.head(10)

In [ ]:
# кластеризуем
distance_matr = squareform(pdist(train.iloc[:, 1:], metric = 'jaccard'))

clust = SpectralClustering()

clst_labels = clust.fit_predict(distance_matr)
labels_set = list(set(clst_labels))
print(labels_set)

In [ ]:
# восстанавливаем значения во всех колонках по месторождениям-аналогам
for dropped_col in dropList_numeric: # числовые
    print("Осредняем по аналогам (числа)")
    subDf = initial_dataset[initial_dataset[dropped_col].isnull()]
    for classy in subDf.groupby('Class')['Class'].unique():
        print("Класс {}".format(classy))
        similar_values_list = initial_dataset[initial_dataset['Class'] == classy[0]][dropped_col]
        if len(similar_values_list) > 1:
            missed_val = similar_values_list.dropna().mean()
            print(missed_val)
            initial_dataset.loc[similar_values_list[similar_values_list.isnull()].index, dropped_col] = missed_val
            
for dropped_col in dropList_categorial: # категориальные
    print("Осредняем по аналогам (категории)")
    subDF = initial_dataset[initial_dataset[dropped_col].isnull()]
    for classy in subDf.groupby('Class')['Class'].unique():
        print("Класс {}".format(classy))
        similar_values_list = initial_dataset[initial_dataset['Class'] == classy[0]][dropped_col]
        missed_val = similar_values_list.value_counts()[similar_values_list.value_counts() == similar_values_list.value_counts().max()].index
        print(missed_val[0])
        initial_dataset.loc[similar_values_list[similar_values_list.isnull()].index, dropped_col] = missed_val[0]

In [ ]:
print(initial_dataset)
initial_dataset.to_csv('recovered_dataset.xlsx')

In [ ]:
def r2(yi_list, fi_list):
    y_mean = sum(yi_list) / float(len(yi_list))
    ss_tot = sum((yi - y_mean) ** 2 for yi in yi_list)
    ss_err = sum((yi - fi) ** 2 for yi, fi in zip(yi_list, fi_list))

In [ ]:
answer = pd.read_csv('dataset_for_check.csv')
response = copy(initial_datset)

In [ ]:
acc = 0
R_2 = 0
if (len(answer[1:]) != len(response[1:])):
    print("Error!")
else:
    acc = accuracy_score(answer['6. Tectonic regime'].values, response['6. Tectonic regime'].values)

porosity_answer = answer['19. Porosity (matrix average %)'].values

porosity_response = response['19. Porosity (matrix average %)'].values

R_2 = r2(porosity_answer, porosity_response)

print(acc, R_2)